In [2]:
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import numpy as np
import tensorflow as tf
import nibabel as nib
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from skimage.transform import resize

import argparse
import logging
import os
import random
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter
from torch.nn.modules.loss import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchvision import transforms

import zipfile
from timeit import default_timer as timer
import matplotlib.pyplot as plt
from copy import deepcopy

from nilearn import plotting

import torch.nn.functional as F

In [3]:
parent_dir = 'datasets/ULS23_DeepLesion3D/categories'

def list_files_in_subfolders(directory):
    subfolders_files = {}
    for entry in os.listdir(directory):
        subfolder_path = os.path.join(directory, entry)
        if os.path.isdir(subfolder_path):
            subfolders_files[entry] = [file[:-4] for file in os.listdir(subfolder_path)]
    return subfolders_files

files = list_files_in_subfolders(parent_dir)

In [10]:
def split_list(data, train_prop=0.6, val_prop=0.2, test_prop=0.2):
    if train_prop + val_prop + test_prop != 1:
        raise ValueError("The proportions must sum up to 1.")
    
    random.shuffle(data)
    
    # Calculate split indices
    total_len = len(data)
    train_end = int(total_len * train_prop)
    val_end = train_end + int(total_len * val_prop)
    
    # Split the data
    train_data = data[:train_end]
    val_data = data[train_end:val_end]
    test_data = data[val_end:]
    
    return train_data, val_data, test_data

train_data, val_data, test_data = [], [], []
for subfolder, filenames in files.items():
    train, val, test = split_list(filenames)
    train_data.extend(train)
    val_data.append(val)
    test_data.append(test)
    
    
def save_list(data, output_dir, filename):
    file_path = os.path.join(output_dir, filename)
    with open(file_path, 'w') as file:
        for item in data:
            file.write(f"{item}\n")
      
output_dir = 'datasets'
save_list(train_data, output_dir, "deeplesion_train.txt")
save_list(val_data, output_dir, "deeplesion_val.txt")
save_list(test_data, output_dir, "deeplesion_test.txt")